In [1]:

from experiments.data_utils import get_uci_data, common_processor_UCI
import numpy as np
import torch

dataset_path = "datasets/UCI_datasets"

    # for dataname in ["boston", "concrete", "energy", "kin8nm","naval", "power", "wine", "yacht"]:
dataname = "boston"

x, y = get_uci_data(data_name= dataname, dir_name= dataset_path)

(train_X_raw, train_Y_raw), (recal_X_raw, recal_Y_raw), (val_X, val_Y), (test_X, test_Y) = common_processor_UCI(x, y)


train_X = np.concatenate((train_X_raw, recal_X_raw), axis = 0)
train_Y = np.concatenate((train_Y_raw, recal_Y_raw), axis = 0)


train_X = torch.Tensor(train_X).to('cuda')
val_X = torch.Tensor(val_X).to('cuda')
test_X = torch.Tensor(test_X).to('cuda')
train_Y = torch.Tensor(train_Y).to('cuda')
val_Y = torch.Tensor(val_Y).to('cuda')
test_Y = torch.Tensor(test_Y).to('cuda')


In [2]:

from experiments.Conformal_Utils.OQR_q_model_ens import QModelEns
from experiments.Conformal_Utils.OQR_losses import batch_qr_loss, batch_interval_loss
import experiments.Conformal_Utils.OQR_helper as helper
from experiments.Conformal_Utils.OQRTrain import OQR_Trainer
from torch.utils.data import DataLoader


dim_x = x.shape[1]
dim_y = 1

model_ens = QModelEns(input_size=dim_x + 1, output_size=dim_y,
                              hidden_size=20, num_layers=2, dropout=0,
                              lr=5e-3, wd=0,
                              num_ens=1, device=torch.device('cuda'))


OQR_Trainer(model_ens, train_X, train_Y, val_X, val_Y, N_Epoch= 30,verbose=True)

        
quantiles = torch.Tensor(np.linspace(0.01,0.99,100))
test_preds = model_ens.predict_q(
    test_X, quantiles, ens_pred_type='conf',
    recal_model=None, recal_type=None
)
print(test_preds.shape)
    


epoch  0
validation loss:  [204.46716309]
epoch  1
validation loss:  [200.88891602]
epoch  2
validation loss:  [195.55926514]
epoch  3
validation loss:  [187.74874878]
epoch  4
validation loss:  [177.3396759]
epoch  5
validation loss:  [162.55160522]
epoch  6
validation loss:  [142.0128479]
epoch  7
validation loss:  [117.28127289]
epoch  8
validation loss:  [96.14016724]
epoch  9
validation loss:  [81.93663788]
epoch  10
validation loss:  [72.63939667]
epoch  11
validation loss:  [61.86957169]
epoch  12
validation loss:  [51.92128372]
epoch  13
validation loss:  [43.53303909]
epoch  14
validation loss:  [37.60130692]
epoch  15
validation loss:  [31.73357582]
epoch  16
validation loss:  [24.08049774]
epoch  17
validation loss:  [19.25227928]
epoch  18
validation loss:  [17.76453018]
epoch  19
validation loss:  [16.52373123]
epoch  20
validation loss:  [15.76578617]
epoch  21
validation loss:  [14.84598255]
epoch  22
validation loss:  [14.08622932]
epoch  23
validation loss:  [13.143082